In [2]:
import pandas as pd
m_df = pd.read_csv('/home/ndo/bash_output/variants_df/varscan_df.csv', sep=",", encoding='utf-8')

In [133]:
m_df
m_df['VAR_CATE'].value_counts()

VAR_CATE
FP    2892193
TP    2011035
FN    1346416
Name: count, dtype: int64

In [134]:
#drop the old FN
m_df.drop(m_df[m_df.VAR_CATE == 'FN'].index, inplace=True)
m_df['VAR_CATE'].value_counts()

VAR_CATE
FP    2892193
TP    2011035
Name: count, dtype: int64

In [135]:
# adding the filter fn
fn = pd.read_csv('/home/ndo/bash_output/05_bedtools_intersect/True_FN.bed', 
            sep='\t', header=None)

fn.rename(columns={0 : 'CHROM', 1 : "START", 2 : "END"}, inplace=True)

In [136]:
fn.drop(columns=['START'], inplace=True)

In [137]:
fn.rename(columns={'END' : 'POS'}, inplace=True)

In [138]:
fn['VAR_CATE'] = 'FN'
fn.value_counts()

CHROM  POS        VAR_CATE
chr16  69659135   FN          24
chr13  34033596   FN          21
chr4   67136      FN          20
chr13  34033607   FN          20
chr20  12140805   FN          16
                              ..
chr2   56752045   FN           1
       56737504   FN           1
       56700243   FN           1
       56681713   FN           1
chr9   138221925  FN           1
Name: count, Length: 540277, dtype: int64

vcf file has 1-based position, purpose is to convert this vcf pos to bed files of left and right flanking sequence (0-based).
from the bed files extract the flanking sequences using bedtools getfasta

In [139]:
m_df = m_df[['CHROM', 'POS','VAR_CATE']]
m_df.value_counts()

CHROM  POS        VAR_CATE
chr10  133668408  FP          3
       133668332  FP          3
       125895968  FP          3
       133668414  FP          3
       133668351  FP          3
                             ..
chr16  10242512   TP          1
       10242017   FP          1
       10236247   TP          1
       10235844   FP          1
chr9   138222748  FP          1
Name: count, Length: 4899587, dtype: int64

In [140]:
# combine the two df

df = pd.concat([m_df, fn])



In [141]:
df.value_counts()

CHROM  POS        VAR_CATE
chr16  69659135   FN          24
chr13  34033596   FN          21
chr4   67136      FN          20
chr13  34033607   FN          20
chr2   132362567  FN          16
                              ..
chr16  50931442   FP           1
       50931418   FP           1
       50930664   TP           1
       50930362   TP           1
chr9   138222748  FP           1
Name: count, Length: 5439864, dtype: int64

In [142]:
#EXTRACT 201 BASES AROUND VARIANT INCLUDING THE CURRENT VARIANT. 100 FROM THE LEFT AND 100 TO THE RIGHT


# left_df = df.copy(deep=True)


# left_df['START'] = left_df['POS'] - 101 #start is incluive
# left_df['END'] = left_df['POS']
# left_df.drop(['POS'], axis=1, inplace=True)
# left_df

In [143]:
#EXTRACT 201 BASES AROUND VARIANT INCLUDING THE CURRENT VARIANT. 100 FROM THE LEFT AND 100 TO THE RIGHT

#convert the current base position to 0-indexed accorind to BED file

df['POS']  = df['POS'] - 1
df['START'] = df['POS'] - 99 # start pos in BED file is inclusive
df['END'] = df['POS'] + 101

df.drop(['POS'], axis=1, inplace=True)
df


,CHROM,VAR_CATE,START,END
0,chr1,FP,601336,601536
1,chr1,FP,601567,601767
2,chr1,FP,604285,604485
3,chr1,FP,604431,604631
4,chr1,FP,604626,604826
...,...,...,...,...
1071867,chr9,FN,138216261,138216461
1071868,chr9,FN,138216453,138216653
1071869,chr9,FN,138217346,138217546
1071870,chr9,FN,138217346,138217546


In [144]:
# right_df = df.copy(deep=True)
# right_df['START'] = right_df['POS'] #start is incluive - 1 for bed file and  +1 for next nucleotide
# right_df['END'] = right_df['POS'] + 100
# right_df.drop(['POS'], axis=1, inplace=True)

# right_df


In [145]:
#reorder the columns postion

desired_order = ['CHROM', 'START', 'END', 'VAR_CATE']
df = df.reindex(columns=desired_order)

In [146]:
tp = df[df['VAR_CATE'] == "TP"]
fp = df[df['VAR_CATE'] == "FP"]
fn = df[df['VAR_CATE'] == "FN"]


In [147]:
# left_df.to_csv('/home/ndo/bash_output/left_flanking/left_flanking.bed', sep='\t', index=False, header=False)
# right_df.to_csv('/home/ndo/bash_output/right_flanking/right_flanking.bed', sep='\t', index=False, header=False)
tp.to_csv('/home/ndo/bash_output/flanking/tp.bed', sep='\t', index=False, header=False)
fp.to_csv('/home/ndo/bash_output/flanking/fp.bed', sep='\t', index=False, header=False)
fn.to_csv('/home/ndo/bash_output/flanking/fn.bed', sep='\t', index=False, header=False)


In [148]:
tp

,CHROM,START,END,VAR_CATE
29,chr1,783075,783275,TP
32,chr1,784760,784960,TP
33,chr1,785317,785517,TP
42,chr1,797292,797492,TP
53,chr1,803946,804146,TP
...,...,...,...,...
6249602,chr9,138212410,138212610,TP
6249610,chr9,138214429,138214629,TP
6249613,chr9,138214818,138215018,TP
6249632,chr9,138217768,138217968,TP


In [149]:

%%bash

bedtools getfasta -tab \
    -fo /home/ndo/bash_output/flanking/tp_flanking.txt  \
    -fi /home/ndo/GIABv3Ref/GRCh38_GIABv3_no_alt_analysis_set_maskedGRC_decoys_MAP2K3_KMT2C_KCNJ18.fasta \
    -bed /home/ndo/bash_output/flanking/tp.bed

bedtools getfasta -tab \
    -fo /home/ndo/bash_output/flanking/fp_flanking.txt  \
    -fi /home/ndo/GIABv3Ref/GRCh38_GIABv3_no_alt_analysis_set_maskedGRC_decoys_MAP2K3_KMT2C_KCNJ18.fasta \
    -bed /home/ndo/bash_output/flanking/fp.bed

bedtools getfasta -tab \
    -fo /home/ndo/bash_output/flanking/fn_flanking.txt  \
    -fi /home/ndo/GIABv3Ref/GRCh38_GIABv3_no_alt_analysis_set_maskedGRC_decoys_MAP2K3_KMT2C_KCNJ18.fasta \
    -bed /home/ndo/bash_output/flanking/fn.bed

In [150]:


# %%bash

# bedtools getfasta -tab \
#     -fo /home/ndo/bash_output/right_flanking/right_flanking.txt  \
#     -fi /home/ndo/GIABv3Ref/GRCh38_GIABv3_no_alt_analysis_set_maskedGRC_decoys_MAP2K3_KMT2C_KCNJ18.fasta \
#     -bed /home/ndo/bash_output/right_flanking/right_flanking.bed

In [3]:
# LOAD the flanking region to df

tp_flank = pd.read_csv('/home/ndo/bash_output/flanking/tp_flanking.txt', sep="\t", header=None, encoding='utf-8')
fp_flank = pd.read_csv('/home/ndo/bash_output/flanking/fp_flanking.txt', sep="\t", header=None, encoding='utf-8')
fn_flank = pd.read_csv('/home/ndo/bash_output/flanking/fn_flanking.txt', sep="\t", header=None, encoding='utf-8')


In [4]:
fn_flank

,0,1
0,chr1:814517-814717,TGTTATTAGTCTCTAATTATAAGAAGATAATCCAGAAGAGAAAGTC...
1,chr1:817414-817614,GCCCTTGGTCCGCTGTGGTTGGGCCTGCACCCAAGCCATGAGCACA...
2,chr1:853776-853976,CCGCCGTGTGGTAAACTGATGAACCCCGACCCTGATCAACATGAGA...
3,chr1:861247-861447,TGGCCTTAGACAGTTCCCTGCCCTTTGTAAGTACTAACTTAACTCT...
4,chr1:873448-873648,TGGGATGGGTACCCTTCTCATTAGTAAGATCCGGGAGGAGTACCCA...
...,...,...
1071867,chr9:138216261-138216461,CCCATGGCAGTGCGACAGCTGCTTCTTGGCAACTCCACCATGATCA...
1071868,chr9:138216453-138216653,TGAATGTTGACAGCACAGGAATCAGGTCATACGGCCCTCCCATCTC...
1071869,chr9:138217346-138217546,AGATTCAGGGCCTGCCCAGGGGCAGAGTGCCCCCAGGACTCGCCCT...
1071870,chr9:138217346-138217546,AGATTCAGGGCCTGCCCAGGGGCAGAGTGCCCCCAGGACTCGCCCT...


**Adding labels to columns**
labels01 labels 0 as TP and everything else is 1 (artifacts). labels012: TP (0), FP (1), FN(2)

In [6]:
tp_flank['labels01'] = tp_flank['labels012'] = 0
tp_flank

,0,1,labels01,labels012
0,chr1:783075-783275,ACTTACATGACCGACATGGTATTTTTATTTGGCAGCGCTGCTCTAA...,0,0
1,chr1:784760-784960,GAAATGAGATATTTATGTAGTCTTAAGGTATCTTCTCACAAATTAT...,0,0
2,chr1:785317-785517,GGGTGACAGGGCATTATATCAGCCACTTATCAACAAACATTTCAGA...,0,0
3,chr1:797292-797492,ACTTTGCTCAATCTTCGCCTTTGGGTTCCTTTAGGTTTATAATGAA...,0,0
4,chr1:803946-804146,TTAGAGGTAACTTTGCTGAACTTGTGAAGAATGGTGTGCAAGGTCC...,0,0
...,...,...,...,...
2011030,chr9:138212410-138212610,CTGTGGAGTGCTAAATCAGAGATTCCCCCAAAGCAGTGGATCAGGA...,0,0
2011031,chr9:138214429-138214629,GTTGCTAGCCAATCGGGACAAATACAGAATGTGAGGTCCCGTTCCA...,0,0
2011032,chr9:138214818-138215018,ACTAGAAGTGAAAAAAAGAATTATTTACTTTGCTGGGAAATGTGAC...,0,0
2011033,chr9:138217768-138217968,TGGTATGACTGCACAGGGATACCTGGCAGGCCAAGTGCAGGGACAG...,0,0


In [7]:
fp_flank['labels01'] = fp_flank['labels012'] = 1
fp_flank

,0,1,labels01,labels012
0,chr1:601336-601536,AGATATGCAGAGTTATTTGATTCTTTCCAGAATCTAATCAAAACAC...,1,1
1,chr1:601567-601767,AGGTCCAGTCCTGTGGGGTTTGACAAACACAGCATCACGTACCCAA...,1,1
2,chr1:604285-604485,CCCAGCTACGCGCAAGGCTGAGGCAGGAGAATGGCGTGAACCCGGG...,1,1
3,chr1:604431-604631,GCAGAGATTGGGTCCCAGCCTGGCTCTGCCTGGCCCTCTGCTCCCG...,1,1
4,chr1:604626-604826,CCTGACCCTGCAGGTCAAGACTTCTGTCTGAGAGAAATGAAAAGCT...,1,1
...,...,...,...,...
2892188,chr9:138221559-138221759,AGAGCCCCAGGGAGACGCGCACCCCACGACTTGCTTTTATTTGCGT...,1,1
2892189,chr9:138221588-138221788,CTTGCTTTTATTTGCGTTCCAGTGGCTCGTTTTTCTTAGAGTGTTA...,1,1
2892190,chr9:138221662-138221862,TGTAGATGTGAGTTTTCACAACCGTATTTTAAGTGTAAATACTGGA...,1,1
2892191,chr9:138221689-138221889,TTTAAGTGTAAATACTGGACTTTGTCTGTTAAAAGGATTTGAGGTA...,1,1


In [8]:
fn_flank['labels01'] = 1
fn_flank['labels012'] = 2
fn_flank

,0,1,labels01,labels012
0,chr1:814517-814717,TGTTATTAGTCTCTAATTATAAGAAGATAATCCAGAAGAGAAAGTC...,1,2
1,chr1:817414-817614,GCCCTTGGTCCGCTGTGGTTGGGCCTGCACCCAAGCCATGAGCACA...,1,2
2,chr1:853776-853976,CCGCCGTGTGGTAAACTGATGAACCCCGACCCTGATCAACATGAGA...,1,2
3,chr1:861247-861447,TGGCCTTAGACAGTTCCCTGCCCTTTGTAAGTACTAACTTAACTCT...,1,2
4,chr1:873448-873648,TGGGATGGGTACCCTTCTCATTAGTAAGATCCGGGAGGAGTACCCA...,1,2
...,...,...,...,...
1071867,chr9:138216261-138216461,CCCATGGCAGTGCGACAGCTGCTTCTTGGCAACTCCACCATGATCA...,1,2
1071868,chr9:138216453-138216653,TGAATGTTGACAGCACAGGAATCAGGTCATACGGCCCTCCCATCTC...,1,2
1071869,chr9:138217346-138217546,AGATTCAGGGCCTGCCCAGGGGCAGAGTGCCCCCAGGACTCGCCCT...,1,2
1071870,chr9:138217346-138217546,AGATTCAGGGCCTGCCCAGGGGCAGAGTGCCCCCAGGACTCGCCCT...,1,2


In [12]:
#concate the three tables:

df_labels = pd.concat([tp_flank, fp_flank, fn_flank])
df_labels

,0,1,labels01,labels012
0,chr1:783075-783275,ACTTACATGACCGACATGGTATTTTTATTTGGCAGCGCTGCTCTAA...,0,0
1,chr1:784760-784960,GAAATGAGATATTTATGTAGTCTTAAGGTATCTTCTCACAAATTAT...,0,0
2,chr1:785317-785517,GGGTGACAGGGCATTATATCAGCCACTTATCAACAAACATTTCAGA...,0,0
3,chr1:797292-797492,ACTTTGCTCAATCTTCGCCTTTGGGTTCCTTTAGGTTTATAATGAA...,0,0
4,chr1:803946-804146,TTAGAGGTAACTTTGCTGAACTTGTGAAGAATGGTGTGCAAGGTCC...,0,0
...,...,...,...,...
1071867,chr9:138216261-138216461,CCCATGGCAGTGCGACAGCTGCTTCTTGGCAACTCCACCATGATCA...,1,2
1071868,chr9:138216453-138216653,TGAATGTTGACAGCACAGGAATCAGGTCATACGGCCCTCCCATCTC...,1,2
1071869,chr9:138217346-138217546,AGATTCAGGGCCTGCCCAGGGGCAGAGTGCCCCCAGGACTCGCCCT...,1,2
1071870,chr9:138217346-138217546,AGATTCAGGGCCTGCCCAGGGGCAGAGTGCCCCCAGGACTCGCCCT...,1,2


In [13]:
#rename the column
df_labels.rename(columns={0: "Chrom:Start-End", 1 : "Flank"})

,Chrom:Start-End,Flank,labels01,labels012
0,chr1:783075-783275,ACTTACATGACCGACATGGTATTTTTATTTGGCAGCGCTGCTCTAA...,0,0
1,chr1:784760-784960,GAAATGAGATATTTATGTAGTCTTAAGGTATCTTCTCACAAATTAT...,0,0
2,chr1:785317-785517,GGGTGACAGGGCATTATATCAGCCACTTATCAACAAACATTTCAGA...,0,0
3,chr1:797292-797492,ACTTTGCTCAATCTTCGCCTTTGGGTTCCTTTAGGTTTATAATGAA...,0,0
4,chr1:803946-804146,TTAGAGGTAACTTTGCTGAACTTGTGAAGAATGGTGTGCAAGGTCC...,0,0
...,...,...,...,...
1071867,chr9:138216261-138216461,CCCATGGCAGTGCGACAGCTGCTTCTTGGCAACTCCACCATGATCA...,1,2
1071868,chr9:138216453-138216653,TGAATGTTGACAGCACAGGAATCAGGTCATACGGCCCTCCCATCTC...,1,2
1071869,chr9:138217346-138217546,AGATTCAGGGCCTGCCCAGGGGCAGAGTGCCCCCAGGACTCGCCCT...,1,2
1071870,chr9:138217346-138217546,AGATTCAGGGCCTGCCCAGGGGCAGAGTGCCCCCAGGACTCGCCCT...,1,2


In [14]:
#save all to cvs file
df_labels.to_csv('/home/ndo/bash_output/data_labels/varscan_labels.csv')

In [ ]:
#next step is to slit the data into train test and validating sets